# RFM Segmentation

Segmenting customers using the R.F.M metric


## Importing Libraries and Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import datetime as dt
import plotly.express as px
# import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv('marketing_campaign.csv', sep = '\t')  
df.head()

## Data Preprocessing:
###  Data Cleaning
To remove errors and modify incorrect input

In [ ]:
def examine_data(data, data_name = 'data'):
    print(f"Examing '{data_name}'")
    print(f'Rows and column shape: {df.shape}\n')
    print(f'{df.info()}\n')


    # duplicate values
    duplicate = df.duplicated().sum()
    if duplicate == 0:
        print(f'Duplicate values:\n No duplicate in the {data_name}.\n')
    else:
        print(f'There are {duplicate} duplicate in the {data_name}.\n')
        
    # missing values
    print('Missing values:')
    missing = pd.DataFrame(df.isna().sum().to_frame().reset_index().\
    rename({'index': 'variables', 0: 'missing_values'}, axis = 1))
    nan_rows = missing[missing['missing_values'] > 0]
    if nan_rows['missing_values'].sum() > 0:
        nan_rows.plot(x = 'variables', y = 'missing_values', kind = 'barh') 
        plt.title('Missing values')
        plt.grid()
    else:
        print(f'There are no missing values in {data_name}.') 
    
examine_data(df, data_name = 'MARKETING DATA')


In [ ]:
# Check income distribution to handle missing rows

# %%
def viz_dist(data, column):
    fig = px.histogram(data, column, 
                       width = 600, 
                       height = 300, 
                       color_discrete_sequence = ['darkblue'])
    fig.show()
    
viz_dist(df, 'Income')

# %% [markdown]
# Handling missing values: Income column is not symmetric, it is skewed, so we cant fill with mean, for the objective of the analysis, customers with missing income value ideally did not input their income or it was not stored, for accuracy we will remove the missing values.

# %%
df.fillna(0, inplace = True)

# %% [markdown]
# Correlation between features in column

# %%
df.corr()
plt.figure(figsize = (30, 15))
sns.heatmap(df.corr(), annot= True);

# %% [markdown]
# Z_CostContact and Z_Revenue as seen in the correlation heatmap have the same values in all its column, these column will be dropped as they do not represent a statistical significant feature to the model.

# %%
df.drop(['Z_CostContact', 'Z_Revenue'], axis = 1, inplace = True)


In [ ]:
df.drop(['Z_CostContact', 'Z_Revenue'], axis = 1, inplace = True)

# %% [markdown]
# ### Data Wrangling
# Feature Engineering: formatting the data into meaningful format for analysis

# %%
# to check the max year of customer erollment with the company for further engineering
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])
max(df['Dt_Customer'])

# %%
# created a new column '2014' to get number of year customers have been erolled
df['2014'] ='2014-12-31'
df['2014'] = pd.to_datetime(df['2014'])
df['customers_engagement'] = (df['2014'].dt.year) - (df['Dt_Customer'].dt.year)  

# age of customers as at the last erollment
df['Age'] = 2014 - df['Year_Birth']

# %%
# total number of kids, whether thier children are kids or teen, altogether they are children
df['Kids'] = df['Kidhome'] + df['Teenhome']
     
# total amount of product purchased
df['total_amount_spent'] = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + df['MntFishProducts'] + df['MntSweetProducts']  + df['MntGoldProds']

# total accepted campaign
df['total_accepted_campaign'] = df['AcceptedCmp3'] + df['AcceptedCmp1'] + df['AcceptedCmp2'] + df['AcceptedCmp4'] + df['AcceptedCmp5']

# total number of purchases made
df['total_num_of_purchase'] = df['NumWebPurchases'] + df['NumCatalogPurchases'] + df['NumStorePurchases'] + df['NumDealsPurchases']

# %%
# delete redundant columns to reduce complexity on model
df.drop(['Year_Birth', 'Dt_Customer', 'Kidhome', 'Teenhome','AcceptedCmp3', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp4', 'AcceptedCmp5','NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
    'NumDealsPurchases', 'NumWebVisitsMonth', 'MntWines', 'MntFruits',
    'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
    '2014', 'Complain', 'Response'], axis = 1, inplace = True)

df.head()

In [ ]:
print(f'Total categories in the feature marital status:\n {df["Marital_Status"].value_counts()},\n')
print(f'Total categories in the feature education:\n {df["Education"].value_counts()}')

# %%
# replace values in column to a more precised category for 'marital_status'
df['Marital_Status'] = df['Marital_Status'].replace(['Married', 'Together'], 'In relationship')

df['Marital_Status'] = df['Marital_Status'].replace(['Single', 'Divorced','Widow', 'Alone', 'YOLO', 'Absurd'], 'Single')

df['Marital_Status'].value_counts()


# replace values in column to a more precised category for 'education'
df.Education = df.Education.replace(['Graduation', 'PhD', 'Master', '2n Cycle'],'PG')
df.Education = df.Education.replace(['Basic'], 'UG')

display(f'New row and column shape: {df.shape}')
df.head()

In [ ]:
# Rename column name into appropriate format

df.columns = df.columns.str.lower()

In [ ]:
# Number of customers in marital and education status

# %%
df_marital_edu_count = df.groupby(['marital_status', 'education'])['id'].count().to_frame().reset_index()
df_marital_edu_count

# %%
def viz_distribution(data, row, column, color, title):
    fig = px.bar(data, row, column,
             color = color, 
             barmode = 'group',
             height = 400,
             width = 600,
             title = title,
             color_discrete_sequence = ['darkblue', 'orange'])
    fig.show()

viz_distribution(df_marital_edu_count, 'marital_status', 'id',  'education', 'Number of customers based on their marital and education status') 


In [ ]:
# Categorising customers based on their relationship and education type to visualise which of the group purchase more

# %%
df_marital_edu_value = df.groupby(['marital_status', 'education'])['total_amount_spent'].sum().to_frame().reset_index()
df_marital_edu_value

# %%
viz_distribution(df_marital_edu_value, 'marital_status', 'total_amount_spent',  'education', 'Customers purchase based on their marital and education status') 

# %%
df_marital = df.groupby('marital_status')['total_amount_spent'].sum().to_frame().reset_index()


In [ ]:


def plot_graph(data,  values, name, color, title):
    fig = px.pie(data, values = values, names = name, title = title, 
                 hole = 0.4, width = 600,
                 color_discrete_sequence = ['darkblue', 'orange'])
    fig.update_traces(textinfo = 'percent', textfont_size = 15)
    fig.show()

plot_graph(df_marital , df_marital['total_amount_spent'], df_marital['marital_status'], df_marital['marital_status'], 'Amount contributed by marital status')

# %%
df_edu = df.groupby('education')['total_amount_spent'].sum().to_frame().reset_index()

plot_graph(df_edu , df_edu['total_amount_spent'], df_edu['education'], df_edu['education'], 'Amount contributed by education')

# %% [markdown]
# ### Observation
# 1. Majority of our customers are post-graduate and in a relationship.
# 2. Customers in relationship and in PG school purchase engrossively than single PG school customers
# 3. Over 64% of customers in relationship purchase more than the single customers.
# 4. Postgraduate customers purchase over 100% of the product more than the undergraduate.
# 
# In summary our best purchasing customers are customers in a relationship and PG school.

# %% [markdown]
# ## Clustering 
# ### Label Encoding


# %%
# as seen above our k=3
km =KMeans(n_clusters=3)
y_predicted= km.fit_predict(df1)
df1['Cluster'] = y_predicted
df1.head()

# %%
# plotting the cluster
PLOT = go.Figure()
for C in list(df1.Cluster.unique()):
    

    PLOT.add_trace(go.Scatter3d(x = df1[df1.Cluster == C]['Income'],
                                y = df1[df1.Cluster == C]['Age'],
                                z = df1[df1.Cluster == C]['Total_expense'],                        
                                mode = 'markers',marker_size = 6, marker_line_width = 1,
                                name = str(C)))
PLOT.update_traces(hovertemplate='Income: %{x} <br>Age: %{y} <br>Total_expense: %{z}')

    
PLOT.update_layout(width = 800, height = 800, autosize = True, showlegend = True,
                   scene = dict(xaxis=dict(title = 'Income', titlefont_color = 'black'),
                                yaxis=dict(title = 'Age', titlefont_color = 'black'),
                                zaxis=dict(title = 'Expense', titlefont_color = 'black')),
                   font = dict(family = "Gilroy", color  = 'black', size = 12))
